# enVector Quick Start

This example demonstrates the complete workflow of **enVector** Python SDK, showcasing its capabilities for encrypted similarity search.

## Import SDK

enVector SDK, `pyenvector` (Encrypted Similarity Search) is a high-performance encrypted vector search engine designed to protect data privacy during similarity search. It enables clients to search over encrypted vectors without ever exposing the raw data or query.

For the first time, we should install and import the SDK package to use enVector Python APIs.
Before installing, make sure Python 3.12 in the virtual environment is installed on your system.
After installation, you can import the SDK in your Python code.

For more details, see [SDK installation](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/get-started/installation/client-sdk) section in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
import pyenvector as ev

## Vector Search

### 1. Initialize

To use the enVector service, initialization is required. Here, we focus on the minimal steps, including:

1) establishing a connection to the enVector server with `address` and `access_token`, 
2) registering a key for enabling enVector server with `key_path` and `key_id` to perform secure operations automatically.

For more details, see [Initialize](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/initialize) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
ev.init(
    address="localhost:50050",
    # access_token="...", # if needed
    key_path="./keys",
    key_id="quickstart_key",
)

### 2. Create Index

To insert data into the Index, we first need to create an index. An index is defined by its name and the dimensionality of the vectors to be stored. The dimensionality must match the size of the vectors planned to insert. For example, if the vectors have 512 dimensions, the index should be created with the specified dimension `dim=512`. This step ensures the index is properly configured to handle the client's data.

For more details, see [Creation](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/encrypted-index/creation) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
index = ev.create_index("quickstart_index", dim=512)

### 3. Insert Data

To populate the index, we need to insert data into the created index. For this example, we will use randomly generated vectors as test data. Each vector should match the dimensionality specified during index creation. Additionally, metadata can be attached to each vector to provide the corresponding context or additional information. This step demonstrates how to prepare and insert sample vector data into the index for testing purposes.

For more details, see [Insert](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/insert) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
import numpy as np

vecs = np.random.rand(10, 512)
vecs = vecs / np.linalg.norm(vecs, axis=1, keepdims=True)  # normalize for IP
metadata = [f"Item {i+1}" for i in range(10)]

index.insert(vecs, metadata)

### 4. Encrypted Similarity Search

To perform a similarity search, we can use a query vector to find the most similar vectors in the index. The `index` object contains the decryption key, enabling the enVector server to return encrypted scores. These scores are decrypted by the client-owned keys to retrieve the top-k results along with their indices. This process ensures secure and efficient similarity search operations, even when working with encrypted data.

For more details, see [Search](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/search) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
search_index = ev.Index("quickstart_index")
query = vecs[0]
results = search_index.search(query, top_k=3, output_fields=["metadata"])[0]
print(results)

### Clean Up

We can delete the created index and the registered key when they are no longer needed.

In [ ]:
ev.drop_index("quickstart_index")

In [ ]:
ev.delete_key("quickstart_key")